### 1. 设置proxy

In [9]:
# 设置代理端口
import os
os.environ["http_proxy"] = "http://127.0.0.1:10809"
os.environ["https_proxy"] = "http://127.0.0.1:10809"

# 测试下能否访问外网
import requests
response = requests.get("http://www.google.com")
print(response.status_code)

200


### 2. 尝试获取数据

In [10]:
!python d:/workspace/qlib/scripts/get_data.py qlib_data --target_dir ~/.qlib/qlib_data/cn_data --region cn


2025-09-10 21:44:52.861 | WARNING  | qlib.tests.data:download:62 - The data for the example is collected from Yahoo Finance. Please be aware that the quality of the data might not be perfect. (You can refer to the original data source: https://finance.yahoo.com/lookup.)
2025-09-10 21:44:52.862 | INFO     | qlib.tests.data:download:65 - 20250910214450_qlib_data_cn_1d_latest.zip downloading......

100%|█████████▉| 196258816/196549189 [01:35<00:00, 468324.66it/s]
196549632it [01:35, 2063483.57it/s]                              
2025-09-10 21:46:28.118 | WARNING  | qlib.tests.data:_unzip:124 - will delete the old qlib data directory(features, instruments, calendars, features_cache, dataset_cache): C:\Users\user\.qlib\qlib_data\cn_data
2025-09-10 21:46:28.119 | INFO     | qlib.tests.data:_unzip:128 - C:\Users\user\.qlib\qlib_data\cn_data\20250910214450_qlib_data_cn_1d_latest.zip unzipping......

100%|██████████| 31008/31008 [01:51<00:00, 277.57it/s]


In [13]:
!qrun d:/workspace/qlib/examples/benchmarks/LightGBM/workflow_config_lightgbm_Alpha158.yaml

^C


ModuleNotFoundError. CatBoostModel are skipped. (optional: maybe installing CatBoostModel can fix it.)
ModuleNotFoundError. XGBModel is skipped(optional: maybe installing xgboost can fix it).
ModuleNotFoundError.  PyTorch models are skipped (optional: maybe installing pytorch can fix it).
Training until validation scores don't improve for 50 rounds
[20]	train's l2: 0.981377	valid's l2: 0.993566
[40]	train's l2: 0.973811	valid's l2: 0.993977
[60]	train's l2: 0.967152	valid's l2: 0.994648
Early stopping, best iteration is:
[18]	train's l2: 0.982182	valid's l2: 0.993497
'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2017-01-03 SH600000   -0.027139
           SH600008    0.007009
           SH600009    0.009770
           SH600010    0.012275
           SH600015   -0.112909
{'IC': np.float64(0.04680587323833807),
 'ICIR': np.float64(0.3815683918932705),
 'Rank IC': np.float64(0.049049290457489736),
 'Rank ICIR':

[4896:MainThread](2025-09-10 21:55:25,862) INFO - qlib.qrun - [run.py:78] - Render the template with the context: {}
[4896:MainThread](2025-09-10 21:55:25,913) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[4896:MainThread](2025-09-10 21:55:25,948) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[4896:MainThread](2025-09-10 21:55:25,950) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/user/.qlib/qlib_data/cn_data')}
[4896:MainThread](2025-09-10 21:55:26,290) WARNING - qlib.workflow - [expm.py:231] - No valid experiment found. Create a new experiment with name workflow.
[4896:MainThread](2025-09-10 21:55:26,307) INFO - qlib.workflow - [exp.py:258] - Experiment 849225049125133042 starts running ...
[4896:MainThread](2025-09-10 21:55:30,705) INFO - qlib.workflow - [recorder.py:345] - Recorder 139f0971e1c640e08062463775fe04c9 starts running under Experiment 8492

In [14]:
import qlib
from qlib.data import D
from qlib.constant import REG_CN

# Initialize Qlib
qlib.init(provider_uri="~/.qlib/qlib_data/cn_data", region=REG_CN)

# 1. Get 2020 trading days
calendar_2020 = D.calendar(start_time="2020-01-01", end_time="2020-12-31")
print("2020 Trading Days (first 5):", calendar_2020[:5])


[23856:MainThread](2025-09-10 23:17:29,539) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[23856:MainThread](2025-09-10 23:17:31,921) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[23856:MainThread](2025-09-10 23:17:31,925) INFO - qlib.Initialization - [__init__.py:77] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/user/.qlib/qlib_data/cn_data')}


2020 Trading Days (first 5): [Timestamp('2020-01-02 00:00:00') Timestamp('2020-01-03 00:00:00')
 Timestamp('2020-01-06 00:00:00') Timestamp('2020-01-07 00:00:00')
 Timestamp('2020-01-08 00:00:00')]


In [ ]:

# 2. Get CSI 300 instruments (filtered by close price > 20)
from qlib.data.filter import ExpressionDFilter
filter_close = ExpressionDFilter(rule_expression="$close<250")  # Filter: close price > 20
filter_close2 = ExpressionDFilter(rule_expression="$open>200")  # Filter: close price > 20
csi300_instruments = D.instruments(market="csi300", filter_pipe=[filter_close,filter_close2])
csi300_list = D.list_instruments(instruments=csi300_instruments, as_list=True)
print("CSI 300 Instruments (close > 20, first 5):", csi300_list[:5])

In [ ]:

# 3. Get features: close price + 5-day MA for CSI 300 instruments (2020)
features = ["$close", "Mean($close, 5)"]
feature_df = D.features(
    instruments=csi300_instruments,
    fields=features,
    start_time="2020-01-01",
    end_time="2020-01-10",
    freq="day"
)
print("Feature DataFrame (head):")
print(feature_df.head())

In [ ]:
# 4. Calculate custom expression: next-day return for SH600000 (2020)
expr_result = D.expression(
    instrument="SH600000",
    field="Ref($close, -1)/$close - 1",  # Next-day return = (next day close / current close) - 1
    start_time="2020-01-01",
    end_time="2020-01-10"
)
print("Next-day Return for SH600000 (2020-01):")
print(expr_result)